# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [7]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
df = pd.read_csv("output_data/cities_vm.csv")
df.head()

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,mount gambier,-37.824670,140.782007,Rain,73.63,58,1008,11.81
1,dikson,73.508389,80.512382,Clouds,-0.18,95,1010,4.56
2,vaini,-21.192567,-175.177280,Clouds,82.69,74,1014,10.36
3,hermanus,-34.417500,19.236111,Clouds,59.85,66,1018,11.81
4,fare,45.908020,13.516538,Clouds,56.39,96,1020,3.44


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
data = df.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [10]:
mask = (df.wind_speed < 10) & (df.temperature < 80) & (df.temperature > 70) & (df.weather == "Clear")
hotel_df = df.loc[mask].reset_index(drop=True)
hotel_df

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed
0,maracaju,-21.616301,-55.164605,Clear,74.59,44,1009,6.40
1,tessalit,20.201954,1.012453,Clear,71.64,15,1014,9.62
2,nador,35.173992,-2.928120,Clear,70.05,60,1021,8.05
3,fayetteville,36.062584,-94.157433,Clear,74.30,68,1019,8.05
4,xixiang,22.583863,113.879081,Clear,74.30,99,1018,3.09
5,ocampo,27.315022,-102.396727,Clear,72.61,61,1014,8.75
6,cabo san lucas,22.893888,-109.920060,Clear,78.42,60,1006,5.01
7,maragogi,-9.011828,-35.222512,Clear,76.60,82,1015,6.67
8,arlit,18.739079,7.393331,Clear,72.64,18,1013,9.26
9,porbandar,21.640900,69.611000,Clear,77.68,58,1014,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotels = []

for indx, row in hotel_df.iterrows():
    lat = row.latitude
    lng = row.longitude

    # geocoordinates
    target_coordinates = f"{lat}, {lng}"
    target_search = "hotel"
    target_radius = 5000
    target_type = "lodging"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    data = response.json()
    
    if (len(data["results"]) > 0):
        hotel = data["results"][0]["name"]
        address = data["results"][0]["vicinity"]
        rating = data["results"][0]["rating"]
        user_ratings_total = data["results"][0]["user_ratings_total"]
    else:
        hotel = None
        address = None
        rating = np.nan
        user_ratings_total = np.nan
        
    hotel = {"hotel" : hotel,
            "address": address,
            "rating": rating,
            "user_ratings_total": user_ratings_total}
    
    hotels.append(hotel)

In [13]:
df2=pd.DataFrame(hotels)
df2

,hotel,address,rating,user_ratings_total
0,Hotel Pousada Bom Sono,"R. Pereira do Lago, 3271 - Cambarai, Maracaju",4.8,53.0
1,None,None,NaN,NaN
2,Hôtel Mercure Rif Nador,"1 Avenue Ibn Tachfine, Nador",4.3,2570.0
3,Inn at Carnall Hall,"465 Arkansas Ave, Fayetteville",4.4,336.0
4,JW Marriott Hotel Shenzhen Bao'an,"8 Baoxing Rd, Baoan, Shenzhen",4.3,87.0
5,Hotel La Quinta,"Escobedo #156, Centro, Ocampo",4.2,69.0
6,"The Cape, A Thompson Hotel","Carr.Transpeninsular Km.5 - Fracc. A, L-21 Mis...",4.7,1212.0
7,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124 s/n Sitio Carió, M...",4.8,6656.0
8,Hôtel Temet,"P9RX+27R, Arlit",3.8,84.0
9,Lords Inn Porbandar,"near Circuit House Chaupati, GEB Colony, Porba...",4.1,3267.0


In [16]:
hotels_all = pd.merge(hotel_df, df2, left_index=True, right_index=True)
hotels_all

,city,latitude,longitude,weather,temperature,humidity,pressure,wind_speed,hotel,address,rating,user_ratings_total
0,maracaju,-21.616301,-55.164605,Clear,74.59,44,1009,6.40,Hotel Pousada Bom Sono,"R. Pereira do Lago, 3271 - Cambarai, Maracaju",4.8,53.0
1,tessalit,20.201954,1.012453,Clear,71.64,15,1014,9.62,None,None,NaN,NaN
2,nador,35.173992,-2.928120,Clear,70.05,60,1021,8.05,Hôtel Mercure Rif Nador,"1 Avenue Ibn Tachfine, Nador",4.3,2570.0
3,fayetteville,36.062584,-94.157433,Clear,74.30,68,1019,8.05,Inn at Carnall Hall,"465 Arkansas Ave, Fayetteville",4.4,336.0
4,xixiang,22.583863,113.879081,Clear,74.30,99,1018,3.09,JW Marriott Hotel Shenzhen Bao'an,"8 Baoxing Rd, Baoan, Shenzhen",4.3,87.0
5,ocampo,27.315022,-102.396727,Clear,72.61,61,1014,8.75,Hotel La Quinta,"Escobedo #156, Centro, Ocampo",4.2,69.0
6,cabo san lucas,22.893888,-109.920060,Clear,78.42,60,1006,5.01,"The Cape, A Thompson Hotel","Carr.Transpeninsular Km.5 - Fracc. A, L-21 Mis...",4.7,1212.0
7,maragogi,-9.011828,-35.222512,Clear,76.60,82,1015,6.67,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124 s/n Sitio Carió, M...",4.8,6656.0
8,arlit,18.739079,7.393331,Clear,72.64,18,1013,9.26,Hôtel Temet,"P9RX+27R, Arlit",3.8,84.0
9,porbandar,21.640900,69.611000,Clear,77.68,58,1014,6.93,Lords Inn Porbandar,"near Circuit House Chaupati, GEB Colony, Porba...",4.1,3267.0


In [17]:
infos = []

for indx, row in hotels_all.iterrows():
    info_box_template = f"""<dl>
                        <dt>Hotel Name:</dt><dd>{row.hotel}</dd>
                        <dt>Rating:</dt><dd>{row.rating}</dd>
                        <dt>Address:</dt><dd>{row.address}</dd>
                        <dt>Location:</dt><dd>({row.latitude}, {row.longitude})</dd>
                        </dl>
                        """
    infos.append(info_box_template)
    
infos[0]

'<dl>\n                        <dt>Hotel Name:</dt><dd>Hotel Pousada Bom Sono</dd>\n                        <dt>Rating:</dt><dd>4.8</dd>\n                        <dt>Address:</dt><dd>R. Pereira do Lago, 3271 - Cambarai, Maracaju</dd>\n                        <dt>Location:</dt><dd>(-21.6163005, -55.164605)</dd>\n                        </dl>\n                        '

In [18]:
 # get data
data = df.loc[:, ["latitude", "longitude"]]
hotel_data = hotels_all.loc[:, ["latitude", "longitude"]]

# Create BASE, init the graph
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '0px',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type='SATELLITE')

# init the overlay layers
heat = gmaps.heatmap_layer(data, weights=df.humidity)
fig.add_layer(heat)

markers = gmaps.marker_layer(hotel_data, info_box_content=infos)
fig.add_layer(markers)

# show the plot
fig

Figure(layout=FigureLayout(border='0px', height='600px', margin='0 auto 0 auto', padding='1px', width='800px')…